<a href="https://colab.research.google.com/github/sgalda/Labs-2023/blob/main/Ant_Colony_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Algoritmo de colonia de hormigas**

# Introducción

El problema del vendedor viajero (TSP) es un problema clásico de optimización en el que un vendedor debe visitar varias ciudades exactamente una vez y regresar al punto de partida, minimizando la distancia total recorrida. El algoritmo de colonia de hormigas es una técnica heurística bioinspirada que simula el comportamiento de las hormigas en la naturaleza para encontrar soluciones aproximadas a este problema. En este código, se implementa este algoritmo, comenzando con funciones para cargar y procesar datos de entrada, continuando con la simulación de las hormigas y la búsqueda de la mejor ruta.

1. El código define una función llamada `cargar_datos` que toma una ruta de archivo como entrada y se encarga de leer y procesar un archivo TSP (Traveling Salesman Problem). Los archivos TSP contienen información sobre ciudades y sus coordenadas. La función abre el archivo y lee su contenido. Luego, utiliza expresiones regulares (a través del módulo re) para extraer la información pertinente sobre las ciudades. Concretamente, busca patrones que correspondan a un ID de ciudad seguido de dos números (que representan las coordenadas x e y). Posteriormente, procesa esta información y construye un diccionario donde las claves son los IDs de las ciudades y los valores son tuplas con las coordenadas. Finalmente, la función devuelve este diccionario.

In [ ]:
import re  # Importa el módulo de expresiones regulares
import numpy as np  # Importa el módulo de numpy para operaciones matemáticas

def cargar_datos(ruta_archivo):

    # Abre y lee el archivo cuya ruta se pasa como argumento
    with open(ruta_archivo, 'r') as archivo:
        contenido = archivo.read()

    # Define una expresión regular para capturar el ID de la ciudad y sus coordenadas x, y
    patron_ciudad = re.compile(r'(\d+)\s+([\d.e+-]+)\s+([\d.e+-]+)')

    # Busca todas las coincidencias del patrón en el contenido del archivo
    datos_ciudad = patron_ciudad.findall(contenido)

    # Procesa los datos extraídos para construir un diccionario con el ID de la ciudad como clave y las coordenadas como valor
    coord_ciudades = {int(id_ciudad): (float(x), float(y)) for id_ciudad, x, y in datos_ciudad}

    # Devuelve el diccionario construido
    return coord_ciudades


2. La función `calcular_matriz_distancias` toma un diccionario de ciudades y sus coordenadas como entrada y devuelve una matriz que representa las distancias entre cada par de ciudades. Para cada par de ciudades, calcula la distancia euclidiana (como la longitud de la línea recta entre ellos) usando sus coordenadas y guarda esa distancia en la matriz. Luego, la función establece valores extremadamente altos en la diagonal de la matriz, esencialmente diciendo que la "distancia" de una ciudad a sí misma es infinitamente grande (para evitar considerar viajar a la misma ciudad como una opción viable). Finalmente, devuelve la matriz completa de distancias.

In [ ]:
def calcular_matriz_distancias(coord_ciudades):
    num_ciudades = len(coord_ciudades) # Determina la cantidad de ciudades
    matriz_distancias_np = np.zeros((num_ciudades, num_ciudades)) # Inicializa matriz de distancias con ceros

    # Itera sobre cada par de ciudades para calcular la distancia entre ellas
    for i in range(num_ciudades): # Itera sobre la primera ciudad.
        for j in range(i+1, num_ciudades):  # Itera sobre la segunda ciudad (evita repetición)
            x1, y1 = coord_ciudades[i+1] # Obtiene las coordenadas x,y de la primera ciudad
            x2, y2 = coord_ciudades[j+1] # Obtiene las coordenadas x,y de la segunda ciudad

            # Calcula la distancia euclidiana entre las dos ciudades
            distancia = np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

            # Almacena la distancia en la matriz en ambas ubicaciones (i,j) y (j,i)
            matriz_distancias_np[i, j] = distancia
            matriz_distancias_np[j, i] = distancia

    # Establece la diagonal de la matriz a un número grande (1e9). Esto es útil para evitar seleccionar la misma ciudad
    matriz_distancias_np[np.diag_indices_from(matriz_distancias_np)] = 1e9

    return matriz_distancias_np # Devuelve la matriz de distancias


3. La función `costo_ruta`  calcula y devuelve el costo total de recorrer una ruta específica entre ciudades. Toma como entrada una lista ordenada de ciudades (la ruta) y una matriz de distancias entre ciudades. Suma las distancias entre ciudades consecutivas en la ruta y añade la distancia entre la última ciudad y la primera, completando el ciclo del viajante.

In [ ]:
def costo_ruta(ruta, matriz_distancias):
    return sum([matriz_distancias[ruta[i], ruta[i+1]] for i in range(len(ruta)-1)]) + matriz_distancias[ruta[-1], ruta[0]]

4. La función `elegir_proxima_ciudad` decide cuál será la próxima ciudad a visitar, basándose en un conjunto de probabilidades. Toma como entrada la ciudad actual en la que se encuentra, una lista de ciudades que aún no se han visitado y una matriz tau que tiene las probabilidades asociadas a cada posible movimiento. La función calcula las probabilidades de ir a cada ciudad no visitada desde la ciudad actual y, basándose en esas probabilidades, hace una elección aleatoria para determinar la próxima ciudad. Retorna el ID de la ciudad elegida.

In [ ]:
def elegir_proxima_ciudad(ciudad_actual, no_visitadas, tau):
    probabilidades = tau[ciudad_actual, no_visitadas]
    probabilidades = probabilidades / np.sum(probabilidades)
    return np.random.choice(no_visitadas, 1, p=probabilidades)[0]

5. La función `buscar_mejor_ruta` implementa el **algoritmo** para resolver el problema del Viajante de Comercio (TSP). A grandes rasgos, el algoritmo simula el comportamiento de las hormigas al buscar rutas cortas entre alimentos y su nido, utilizando feromonas para marcar y seguir rutas favorables.

In [ ]:
def buscar_mejor_ruta(matriz_distancias_np, alfa, beta, rho, q, max_iteraciones, num_hormigas):

    # Obtiene el número de ciudades a partir de las dimensiones de la matriz de distancias
    num_ciudades = len(matriz_distancias_np)

    # Inicializa la matriz de feromonas con valores uniformes
    matriz_feromonas = np.ones(matriz_distancias_np.shape) / num_ciudades

    # Un pequeño valor para evitar divisiones entre cero
    epsilon = 1e-9

    # Calcula la matriz inicial de tau (probabilidades) usando feromonas y visibilidad (1/distancia)
    tau = matriz_feromonas ** alfa * ((1.0 / (matriz_distancias_np + epsilon)) ** beta)

    # Inicializa la mejor ruta y su costo con valores nulos y infinito, respectivamente
    mejor_ruta = None
    mejor_costo = float('inf')

    # Itera a través de un número máximo de iteraciones
    for iteracion in range(max_iteraciones):
        # Listas para almacenar las rutas y sus costos de todas las hormigas en la iteración actual
        todas_rutas = []
        todos_costos = []

        # Cada hormiga construye su ruta
        for hormiga in range(num_hormigas):
            # Lista de ciudades que aún no han sido visitadas
            no_visitadas = list(range(num_ciudades))

            # Selecciona una ciudad inicial al azar para la hormiga
            ciudad_actual = np.random.choice(no_visitadas)
            no_visitadas.remove(ciudad_actual)
            ruta = [ciudad_actual]

            # Construye la ruta visitando las ciudades una por una
            while no_visitadas:
                # Elige la próxima ciudad basada en las probabilidades de tau
                proxima_ciudad = elegir_proxima_ciudad(ciudad_actual, no_visitadas, tau)
                no_visitadas.remove(proxima_ciudad)
                ruta.append(proxima_ciudad)
                ciudad_actual = proxima_ciudad

            # Guarda la ruta construida y su costo
            todas_rutas.append(ruta)
            coste = costo_ruta(ruta, matriz_distancias_np)
            todos_costos.append(coste)

        # Identifica la ruta con el menor costo en esta iteración
        idx_mejor = np.argmin(todos_costos)
        ruta_mejor_actual = todas_rutas[idx_mejor]
        costo_mejor_actual = todos_costos[idx_mejor]

        # Actualiza la mejor ruta si la nueva es mejor que la mejor ruta conocida
        if costo_mejor_actual < mejor_costo:
            mejor_ruta = ruta_mejor_actual
            mejor_costo = costo_mejor_actual

        # Evapora un porcentaje de feromonas en todas las rutas
        matriz_feromonas *= (1-rho)

        # Actualiza la matriz de feromonas basada en las rutas recién construidas
        for ruta, costo in zip(todas_rutas, todos_costos):
            delta_feromona = q / costo
            for i in range(len(ruta) - 1):
                matriz_feromonas[ruta[i]][ruta[i+1]] += delta_feromona
                matriz_feromonas[ruta[i+1]][ruta[i]] += delta_feromona
            matriz_feromonas[ruta[-1]][ruta[0]] += delta_feromona
            matriz_feromonas[ruta[0]][ruta[-1]] += delta_feromona

        # Recalcula tau (probabilidades) para la siguiente iteración
        tau = matriz_feromonas ** alfa * ((1.0 / (matriz_distancias_np + epsilon)) ** beta)

    # Devuelve la mejor ruta y su costo al finalizar las iteraciones
    return mejor_ruta, mejor_costo

In [ ]:
ruta_archivo = '/content/a280.tsp' # Define la ruta del archivo con los datos del TSP
coord_ciudades = cargar_datos(ruta_archivo) # Carga las coordenadas de las ciudades del archivo
matriz_distancias_np = calcular_matriz_distancias(coord_ciudades) # Calcula matriz de distancias entre ciudades

alfa = 1 # Influencia de feromonas durante construcción de ruta
beta = 5 # Influencia de visibilidad (inversa de distancia)
rho = 0.5 # Tasa de evaporación de feromonas
q = 100 # Cantidad de feromonas depositadas por hormiga
max_iteraciones = 100 # Número máximo de iteraciones del algoritmo
num_hormigas = 50 # Número de hormigas por iteración

# Busca la mejor ruta usando el algoritmo de colonia de hormigas
mejor_ruta, mejor_costo = buscar_mejor_ruta(matriz_distancias_np, alfa, beta, rho, q, max_iteraciones, num_hormigas)

print("Mejor ruta encontrada:", mejor_ruta)
print("Costo de la mejor ruta:", mejor_costo)


Mejor ruta encontrada: [183, 182, 181, 180, 175, 176, 150, 151, 155, 152, 154, 153, 128, 129, 130, 131, 18, 19, 20, 127, 126, 125, 28, 27, 26, 25, 21, 24, 22, 23, 13, 14, 12, 11, 10, 9, 7, 6, 8, 274, 273, 272, 271, 270, 15, 16, 17, 132, 133, 269, 268, 267, 135, 134, 266, 136, 137, 265, 139, 140, 141, 146, 147, 148, 138, 264, 263, 262, 261, 260, 259, 258, 257, 256, 253, 252, 207, 206, 211, 212, 215, 216, 217, 214, 213, 210, 209, 208, 251, 254, 255, 248, 247, 277, 278, 3, 276, 275, 5, 4, 0, 1, 279, 2, 242, 241, 240, 239, 238, 237, 236, 235, 234, 233, 232, 231, 230, 245, 244, 243, 246, 249, 250, 229, 228, 227, 226, 225, 224, 223, 222, 221, 218, 219, 220, 202, 203, 204, 205, 145, 144, 198, 199, 201, 200, 195, 196, 193, 194, 197, 192, 191, 190, 189, 188, 187, 186, 184, 185, 163, 162, 161, 160, 174, 159, 158, 157, 156, 118, 119, 120, 121, 122, 123, 124, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 59, 60, 117, 116, 114, 113, 112, 86, 83, 82, 81, 80, 79, 78, 75, 74, 73, 72, 71, 70,

# Conclusión  

En el proceso de abordar el desafío del problema del viajante (TSP), se implementó una técnica basada en colonias de hormigas. Este enfoque, inspirado en el comportamiento natural de las hormigas al buscar rutas entre su nido y fuentes de alimentos, se adaptó para encontrar rutas óptimas entre ciudades.

A través del código, se diseñaron funciones para cargar y procesar datos, calcular matrices de distancias entre ciudades y, más importante aún, decidir la próxima ciudad a visitar basándose en la feromona depositada por las hormigas anteriores. Esta feromona actúa como una indicación de qué tan favorable es una ruta particular.

A pesar de que no se alcanzó el costo óptimo conocido para la mejor ruta, que es de 2579, se logró un resultado prometedor de 2947. Este resultado cercano al óptimo es testimonio de las modificaciones y ajustes realizados en las variables del algoritmo. Estos ajustes, aunque no se detallan completamente en este informe, fueron esenciales para acercarse al costo objetivo.

Este trabajo subraya el potencial de los algoritmos bioinspirados, como el de colonias de hormigas, en la resolución de problemas complejos de optimización. Aunque se obtuvo un resultado cercano al óptimo, hay margen para futuras mejoras y refinamientos en el algoritmo, lo que podría acercarnos aún más al costo óptimo o, potencialmente, superarlo.